<h1>Description of the Problem and Discussion of the Background</h1>

Deciding where to open a new venue in London (outside Inner City Limits)
London is a great city. But there is also another side to its popularity and appeal. 
London is an expensive place to live and thrive. 
Many businesses want to open a venue here. 
But what about those, who want to start their own business and cannot really afford to open in the City yet?
Where is it best to open a new place? Where will it be cheapest and will have enough people living around to be popular?
Where the competition is not too overwhelming?

All these questions and more will be answered below. The whole process will be carried out as below:

1.a. Name of the London Boroughs, area and population from web scraping

1.b. Filter the boroughs to get the best candidates for the analysis.

1.c. Use Foresquare/google api Data to obtain info about most popular venues.

3.a. Maximize the number of clusters.

3.b. Visualization using Chloropleth Map

First time enterpreneurs, who want to start their first business. Below dataset will give a comprehensive insight into where best to open a new venue, to maximise the value for money.

People who already run a business and want to branch out. Given the extra information, it may provide valuable information before decision making.

Initial Data Preparation (Week 1) 
2.1. Get The Names of Boroughs, Areas, Population, Coordinates from Wikipedia 
2.2. Processing the Information From Wiki To Make Necessary Lists 
First, we get the nessessary information on London Boroughs, dropping the extras, that will not be needed for the analysis.

<h1> Methodology </h1>
In this project we will direct our efforts on detecting areas of London that have low pub density, keeping an eye on population and rent paid.

In first step we have collected the required **data: location and type (category) of every restaurant within London center** . 
 We have used Foursquare to detect other available entertainment areas available in the region.(according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**entertainment areas**' across different areas of London - we will use **Folium** to identify a few promising areas close to center with low number of restaurants in general (*and* no Italian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** using k-means cluster. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final exploration and search for optimal venue location by population/rent etc.

In [1]:
! pip install geopy
! pip install geocoder
import geopy
import geocoder
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
url='https://en.wikipedia.org/wiki/List_of_London_boroughs'

LDF=pd.read_html(url, header=0)[0]

LDF.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2019 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp...",25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [3]:

LF = LDF.drop(['Status','Local authority','Political control','Headquarters','Nr. in map'], axis=1)
LF['Inner'].replace(np.nan,'0', inplace=True)
LF['Borough'].replace('Barking and Dagenham [note 1]','Barking and Dagenham', inplace=True)
LF['Borough'].replace('Greenwich [note 2]','Greenwich', inplace=True)
LF['Borough'].replace('Hammersmith and Fulham [note 4]','Hammersmith and Fulham', inplace=True)
Inn = ['Camden','Greenwich','Hackney','Hammersmith and Fulham','Islington','Kensington and Chelsea','Lewisham','Lambeth','Southwark','Tower Hamlets','Wandsworth','Westminster']
LF.head()
LF['Inner'] = '0'
LF.head()

,Borough,Inner,Area (sq mi),Population (2019 est)[1],Co-ordinates
0,Barking and Dagenham,0,13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp..."
1,Barnet,0,33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,0,23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,0,16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,0,57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E


Renaming some of the columns

In [4]:

LF['Inner'] = LF.Borough.isin(Inn).astype(int)
Out = LF[LF.Inner == 0]
Out = Out.drop(['Inner'], axis=1)
df = Out.rename(columns = {"Area (sq mi)": "Area", 
                            "Population (2013 est)[1]":"Population"})
geolocator = Nominatim(user_agent="London_explorer")
df['Co-ordinates']= df['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df[['Latitude', 'Longitude']] = df['Co-ordinates'].apply(pd.Series)
df.head()

,Borough,Area,Population (2019 est)[1],Co-ordinates,Latitude,Longitude
0,Barking and Dagenham,13.93,212906,"(51.5541171, 0.15050434261994267)",51.554117,0.150504
1,Barnet,33.49,395896,"(51.65309, -0.2002261)",51.653090,-0.200226
2,Bexley,23.38,248287,"(39.9692378, -82.936864)",39.969238,-82.936864
3,Brent,16.70,329771,"(32.9373463, -87.1647184)",32.937346,-87.164718
4,Bromley,57.97,332336,"(51.4028046, 0.0148142)",51.402805,0.014814


Rent price is a big issue for the new business, we will get the boroughs with the lowest ones.

In [5]:
Max_Rent = ['102.25','150.75','97','150.75','118.5','129.25','140','102.25','107.75','140','86','161.5','161.5','140','123.75','134.5','118.5','140','129.25','145.25']
df['Max_Rent'] = Max_Rent

df["Max_Rent"] = pd.to_numeric(df["Max_Rent"])
fin = df[df.Max_Rent <= 125]
fin

Long_list = fin['Longitude'].tolist()
Lat_list = fin['Latitude'].tolist()
print ("Old latitude list: ", Lat_list)
print ("Old Longitude list: ", Long_list)

replace_longitudes = {-106.6621329:0.0799, -2.8417544: 0.1837}
replace_latitudes = {50.7164496:51.6636, 51.0358628: 51.5499}

longtitudes_new = [replace_longitudes.get(n7,n7) for n7 in Long_list]
latitudes_new = [replace_latitudes.get(n7,n7) for n7 in Lat_list]

fin = fin.drop(['Co-ordinates', 'Longitude'], axis=1)

fin['Longitude'] = longtitudes_new
fin['Latitude'] = latitudes_new
fin

Old latitude list:  [51.5541171, 39.9692378, 51.4028046, 51.6520851, 51.587929849999995, 51.0043613, 51.41086985, 51.5763203]
Old Longitude list:  [0.15050434261994267, -82.936864, 0.0148142, -0.0810175, -0.10541010599099046, -2.337474942629507, -0.18809708858824303, 0.0454097]


,Borough,Area,Population (2019 est)[1],Latitude,Max_Rent,Longitude
0,Barking and Dagenham,13.93,212906,51.554117,102.25,0.150504
2,Bexley,23.38,248287,39.969238,97.00,-82.936864
4,Bromley,57.97,332336,51.402805,118.50,0.014814
8,Enfield,31.74,333794,51.652085,102.25,-0.081018
12,Haringey,11.42,268647,51.587930,107.75,-0.105410
14,Havering,43.35,259552,51.004361,86.00,-2.337475
22,Merton,14.52,206548,51.410870,123.75,-0.188097
24,Redbridge,21.78,305222,51.576320,118.50,0.045410


In [6]:

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'London'

geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
London_latitude = location.latitude
London_longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(London_latitude, London_longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


In [7]:
!pip install folium
import folium

Fin_Brgh = folium.Map(location=[London_latitude, London_longitude], zoom_start=12)


for lat, lng, label in zip(fin['Latitude'], fin['Longitude'], 
                            fin['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#Blue',
        fill_opacity=0.7).add_to(Fin_Brgh)
Fin_Brgh

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [8]:

CLIENT_ID = 'RVOHL5S50AJKSI1KC5Y5XJPMGF3R2XWDYO30GF3SNHVNWQAO' #'your-client-ID' # your Foursquare ID
CLIENT_SECRET = '4KX1YG2NIBDEISK3B0MP5LE0DIEKNZIXUSRH04ZYKYI4RLLW' #'your-client-secret' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('My credentails:')
print('My CLIENT_ID: ' + CLIENT_ID)
print('My CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
My CLIENT_ID: RVOHL5S50AJKSI1KC5Y5XJPMGF3R2XWDYO30GF3SNHVNWQAO
My CLIENT_SECRET:4KX1YG2NIBDEISK3B0MP5LE0DIEKNZIXUSRH04ZYKYI4RLLW


Due to a big borough area, we will take a rather big radius of search.

In [9]:
radius = 5000
LIMIT = 100

def getVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue_Lat', 
                  'Venue_Long', 
                  'Venue_Category']
    
    return(nearby_venues)


In [10]:
Brgh_Venues = getVenues(names=fin['Borough'],
                        latitudes=fin['Latitude'],
                        longitudes=fin['Longitude'])

Barking and Dagenham
Bexley
Bromley
Enfield
Haringey
Havering
Merton
Redbridge


Lets count the venues in every borough, to get the data scale

In [11]:
Brgh_Venues.groupby('Borough').count()

,Latitude,Longitude,Venue,Venue_Lat,Venue_Long,Venue_Category
Borough,,,,,,
Barking and Dagenham,96,96,96,96,96,96
Bexley,96,96,96,96,96,96
Bromley,100,100,100,100,100,100
Enfield,100,100,100,100,100,100
Haringey,100,100,100,100,100,100
Havering,4,4,4,4,4,4
Merton,100,100,100,100,100,100
Redbridge,100,100,100,100,100,100


Lets see which venues are more popular than the others

In [12]:
London_Brgh_onehot = pd.get_dummies(Brgh_Venues[['Venue_Category']], prefix="", prefix_sep="")
mid =  Brgh_Venues['Borough']

London_Brgh_onehot.insert(0, 'Borough', mid)

London_Brgh_onehot.head()

,Borough,ATM,Airfield,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [17]:
Brgh_grouped = London_Brgh_onehot.groupby('Borough').mean().reset_index()
Brgh_grouped

,Borough,ATM,Airfield,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Barking and Dagenham,0.000000,0.00,0.010417,0.00,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.010417,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
1,Bexley,0.010417,0.00,0.020833,0.00,0.00,0.010417,0.010417,0.00,0.010417,...,0.00,0.010417,0.00,0.020833,0.010417,0.00,0.00,0.010417,0.010417,0.00
2,Bromley,0.000000,0.00,0.010000,0.00,0.00,0.000000,0.010000,0.01,0.000000,...,0.00,0.020000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.01
3,Enfield,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.01,0.000000,...,0.01,0.070000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
4,Haringey,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.060000,0.01,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.00
5,Havering,0.000000,0.25,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00
6,Merton,0.000000,0.00,0.000000,0.02,0.00,0.000000,0.020000,0.00,0.000000,...,0.00,0.000000,0.00,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.00
7,Redbridge,0.000000,0.00,0.000000,0.00,0.02,0.000000,0.000000,0.00,0.000000,...,0.00,0.020000,0.02,0.000000,0.000000,0.01,0.01,0.000000,0.000000,0.00


In [18]:
num_top_venues = 5

for brgh in Brgh_grouped['Borough']:
    print("_________"+brgh+"________")
    temp = Brgh_grouped[Brgh_grouped['Borough'] == brgh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

_________Barking and Dagenham________
           venue  freq
0  Grocery Store  0.10
1           Park  0.09
2    Supermarket  0.09
3    Coffee Shop  0.08
4            Pub  0.06


_________Bexley________
                venue  freq
0         Coffee Shop  0.07
1         Pizza Place  0.05
2  Chinese Restaurant  0.04
3      Ice Cream Shop  0.04
4                Park  0.03


_________Bromley________
                  venue  freq
0                   Pub  0.11
1                  Park  0.08
2           Coffee Shop  0.08
3         Grocery Store  0.06
4  Gym / Fitness Center  0.05


_________Enfield________
                venue  freq
0                 Pub  0.09
1         Coffee Shop  0.09
2  Turkish Restaurant  0.07
3                Café  0.06
4         Supermarket  0.05


_________Haringey________
                venue  freq
0                 Pub  0.12
1                Café  0.11
2         Coffee Shop  0.09
3                Park  0.07
4  Turkish Restaurant  0.06


_________Havering________
    

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
brgh_venues_sorted = pd.DataFrame(columns=columns)
brgh_venues_sorted['Borough'] = Brgh_grouped['Borough']

for ind in np.arange(Brgh_grouped.shape[0]):
    brgh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brgh_grouped.iloc[ind, :], num_top_venues)

brgh_venues_sorted.head(8)

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,Grocery Store,Supermarket,Park,Coffee Shop,Pub
1,Bexley,Coffee Shop,Pizza Place,Ice Cream Shop,Chinese Restaurant,Bakery
2,Bromley,Pub,Coffee Shop,Park,Grocery Store,Pizza Place
3,Enfield,Pub,Coffee Shop,Turkish Restaurant,Café,Park
4,Haringey,Pub,Café,Coffee Shop,Park,Turkish Restaurant
5,Havering,Hotel,Airfield,Food & Drink Shop,Electronics Store,Fast Food Restaurant
6,Merton,Pub,Park,Coffee Shop,Café,Bar
7,Redbridge,Pub,Park,Coffee Shop,Restaurant,Café


In [20]:

kclusters = 3

brgh_grouped_clustering = Brgh_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brgh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 0, 0, 0, 1, 0, 0], dtype=int32)

In [21]:
brgh_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)
Borough_merged = pd.merge(fin,brgh_venues_sorted, on='Borough')
Borough_merged

,Borough,Area,Population (2019 est)[1],Latitude,Max_Rent,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,13.93,212906,51.554117,102.25,0.150504,2,Grocery Store,Supermarket,Park,Coffee Shop,Pub
1,Bexley,23.38,248287,39.969238,97.00,-82.936864,2,Coffee Shop,Pizza Place,Ice Cream Shop,Chinese Restaurant,Bakery
2,Bromley,57.97,332336,51.402805,118.50,0.014814,0,Pub,Coffee Shop,Park,Grocery Store,Pizza Place
3,Enfield,31.74,333794,51.652085,102.25,-0.081018,0,Pub,Coffee Shop,Turkish Restaurant,Café,Park
4,Haringey,11.42,268647,51.587930,107.75,-0.105410,0,Pub,Café,Coffee Shop,Park,Turkish Restaurant
5,Havering,43.35,259552,51.004361,86.00,-2.337475,1,Hotel,Airfield,Food & Drink Shop,Electronics Store,Fast Food Restaurant
6,Merton,14.52,206548,51.410870,123.75,-0.188097,0,Pub,Park,Coffee Shop,Café,Bar
7,Redbridge,21.78,305222,51.576320,118.50,0.045410,0,Pub,Park,Coffee Shop,Restaurant,Café


In [22]:
map_clusters = folium.Map(location=[London_latitude, London_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, rent, pop in zip(Borough_merged['Latitude'],
                                  Borough_merged['Longitude'],
                                  Borough_merged['Borough'],
                                  Borough_merged['Cluster Label'],
                                  Borough_merged['Max_Rent'],
                                  Borough_merged['Population (2019 est)[1]']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + " " + "Rent " + str(rent) + " " + "Population " + str(pop), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=25,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [23]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 0, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Borough,Area,Population (2019 est)[1],Max_Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Bromley,57.97,332336,118.50,0,Pub,Coffee Shop,Park,Grocery Store,Pizza Place
3,Enfield,31.74,333794,102.25,0,Pub,Coffee Shop,Turkish Restaurant,Café,Park
4,Haringey,11.42,268647,107.75,0,Pub,Café,Coffee Shop,Park,Turkish Restaurant
6,Merton,14.52,206548,123.75,0,Pub,Park,Coffee Shop,Café,Bar
7,Redbridge,21.78,305222,118.50,0,Pub,Park,Coffee Shop,Restaurant,Café


These boroughs already have a pub.Hence it would not be ideal to open another pub a it might lead to stiff competition.However even if one decides to open a pub here,then Enfield can be a good option followed by Haringey.

In [24]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 1, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Borough,Area,Population (2019 est)[1],Max_Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Havering,43.35,259552,86.0,1,Hotel,Airfield,Food & Drink Shop,Electronics Store,Fast Food Restaurant


Havering does not seem to have a pub.The rent is also low.Hence it can be a good option to start a pub in this locality.

In [25]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 2, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]

,Borough,Area,Population (2019 est)[1],Max_Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,13.93,212906,102.25,2,Grocery Store,Supermarket,Park,Coffee Shop,Pub
1,Bexley,23.38,248287,97.00,2,Coffee Shop,Pizza Place,Ice Cream Shop,Chinese Restaurant,Bakery


Bexley is a good fit to our target boroughs: Rent is low,decent population and while there are already a lot of entertainment areas,it seem to lack a pub.

<h1>Results</h1>
Results of the above analysis and clustering cam be summarized:

The most popular social venues, ouside of Inner London boroughs are Pubs and Coffee shops
Northern boroughs are more prone to visiting pubs, whereas southern boroughs are most likely to shop and have the social life from home
Within top 5 places of interest in every borough is an ethnic restaurant
Rent price is not so much a factor for going out - the demand is not affected by difference in costs

<h2> Discussion</h2>

This notebook is highly inspired by the template given in the course. The idea of clustering the city by area and then plot heatmap to find better area.

Some details on the data used:

Country/City: London
Data: Source is Wikipedia 
Goal: Open a pub in London based on cost/population and other entertainment zones
The following API has been used

Foursquare API: to find restaurant/venues


<h1> Conclusion </h1>
This project can be reused for other cities, just think about changing clustering size to adapt to your city.

It's very far from being perfect, a lot of work can be done, other source of data can be found, but in the end the result seams to correlate with the real world, when we know the city, the area predicted seams correct.